In [2]:
# from google.colab import drive
# drive.mount('/content/drive/')

Mounted at /content/drive/


# Задача nearest deduplication

Задача nearest deduplication (ближайшей дедупликации) заключается в нахождении и устранении дубликатов данных, основываясь на некотором расстоянии или схожести между элементами. Эта задача часто встречается в областях обработки текстов, изображений, звука и других типов данных, где важно идентифицировать и удалять повторяющиеся или почти идентичные элементы.



Nearest deduplication решается, например, в задачах обработки текстов, когда вам нужно произвести:

- Удаление дубликатов статей, документов или записей в базе данных;
- Удаление дубликатов новостей, с которыми, например, потом работают люди из отдела рисков в банках;
- Идентификация похожих отзывов или комментариев в системах обратной связи;


Считаем данные необходимые для работы

Классически, задачу можно решать используя модель:

1. Эмбеддингов, производя сементический поиска;
2. Дообучим свою модель cross encoder;
3. Дообучив модель bi-encoder;
4. Добавить к bi-encoder модель reranker.

![Cross-Encoder+BiEncoder](https://raw.githubusercontent.com/UKPLab/sentence-transformers/master/docs/img/Bi_vs_Cross-Encoder.png)

В нашем случае мы ограничимся моделями из 2-го и 3-го пунктов.

Если коротко говорить про модель bi-encoder, то принцип ее работы следующий;

1. Есть "текст 1" и "текст 2";
2. Получаем "эмбеддинг 1" и "эмбеддинг 2";
3. Считаем cosine-similarity;
4. Получаем скор;

Если коротко говорить про модель cross-encoder, то принцип ее работы следующий:

1. Есть "текст 1" и "текст 2";
2. Отправляем их в одну модель одновременно;
3. Получаем скор.

В чем их основное отличие? - В том, что bi-encoder выдает предварительно эмбеддинги текстов, а cross-encoder - нет и из-за этого он является более тяжелым, по-скольку ему необходимо больше сравнений.

Поэтому работу bi-encoder и cross encoder комбинируют следующим образом:

1. Сначала мы быстрым поиском ищем объекты при помощи bi-encoder;
2. Получаем скоры текстов;
3. Отбираем top-k текстов по скорам;
4. Далее производим переранжирование при помощи cross-encoder;
5. Оставляет top-p объектов после переранжирования.

In [6]:
!pip install hdbscan
!pip install sentence_transformers
!pip install cugraph
!pip install faiss-gpu
!pip install datasets
!pip install cugraph-cu12 --extra-index-url=https://pypi.nvidia.com

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 69.9 MB/s eta 0:00:00
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.10
    Uninstalling Cython-3.0.10:
      Successfully uninstalled Cython-3.0.10
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)


In [28]:
from typing import Dict, List
import math
import os
from datetime import datetime

import torch
from torch import nn
from torch.utils.data import DataLoader
from sentence_transformers import models, losses, evaluation, LoggingHandler, SentenceTransformer
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator
from sentence_transformers.readers import InputExample
from sentence_transformers import SentenceTransformer, util

import logging
from transformers import logging as lg

import data_loaders, clu_evaluators
import pandas as pd


lg.set_verbosity_error()
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
logger = logging.getLogger(__name__)


In [8]:
path_train_data: str = 'train_set.csv'
path_val_data: str = 'dev_set.csv'

df_train: pd.DataFrame = pd.read_csv(path_train_data, sep='\t', index_col='Unnamed: 0').sample(frac=0.2)
df_test: pd.DataFrame = pd.read_csv(path_val_data, sep='\t', index_col='Unnamed: 0').sample(frac=0.2)

Классически эта задача может решаться при помощи хэш-алгоритмов. Таких, например, как LSH, но мы с вами рассмотрим нейросетевой подход. Давайте для начала напишем функцию, которая предобработает загруженные данные в нужный формат

In [9]:
def extract_raw_data(raw_data: pd.DataFrame) -> Dict[str, List]:

    sentence_1_list = [str(i) for i in list(raw_data["Text 1"])]
    sentence_2_list = [str(i) for i in list(raw_data["Text 2"])]
    labels = list(raw_data["Label"])

    return {'sentence_1': sentence_1_list, 'sentence_2': sentence_2_list, "labels": labels}

In [10]:
train_data = extract_raw_data(df_train)
dev_data = extract_raw_data(df_test)

In [16]:
output_id: int = 1

sentence_1_example = dev_data['sentence_1'][output_id]
sentence_2_example = dev_data['sentence_2'][output_id]
sentence_label_example = dev_data['labels'][output_id]

print(f"# Первый текст: {sentence_1_example}")
print(f"# Предполагаемый дубликат: {sentence_2_example}")
print(f"# Класс: {sentence_label_example}")

# Первый текст: New York, March 7.-—(AP)--The
tueboat Joyce Card blew up as she
was moving out of Erie Basin,
ttrooklyn, today, killing two men
and injuring three. Three others
are missing.

The boat, owned by the Card
Towing Line, Inc., had just been
overhauled and was starting out on
her first assignment when a blast
in the boiler room sent her to the
bottom.

The three men rescued were on
the deck at the time of the ex-
plosion, They were picked up by
rescuers who dived from other tugs
nearby, The rest of the crew was
below deck and it was believed
certain that all died. Two bodies
were recovered,
 
# Предполагаемый дубликат: NEW YORK, March T—(AP)—
The tugboat, Joyce Card, blew up
taday, killing two men and injur-
ing three. Three others are miss-
fy.

A blast in the boiler room sent
her Lo the bottom.

The three men reseued were on
the deck at the time of the explo-
sion. They were picked tp hy res-
utys. The r of the crew was
below and it was helieved all were
dead. Two bodies we

# Напишем функцию для тренировки bi-encoder

In [13]:
base_model='sentence-transformers/all-mpnet-base-v2'
add_pooling_layer=False
train_batch_size=4
num_epochs=2
warmup_epochs=2
loss_fn='contrastive'
loss_params={'distance_metric': losses.SiameseDistanceMetric.COSINE_DISTANCE, 'margin': 0.2}
model_save_path=f'output/{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}'

Прежде всего инициализируем трансформерную модель в обертках SentenceTransformer.


**Зачем здесь add_pooling_layer и для чего он может быть нужен?**

Добавление слоя Pooling в данном контексте выполняет несколько важных функций при работе с моделями обработки естественного языка (NLP).

Основные цели добавления Pooling слоя
Создание фиксированного размерного представления предложений:

1. Входные данные могут быть разной длины, а модели, такие как трансформеры, производят эмбеддинги для каждого отдельного токена в предложении.
2. Для многих задач, таких как классификация предложений, нахождение сходства между предложениями и другие, требуется фиксированное размерное представление всего предложения.
Pooling слой объединяет эмбеддинги отдельных токенов в одно фиксированное размерное представление, что упрощает дальнейшую обработку и анализ.

**Снижение размерности:**

1. Вместо того чтобы работать с эмбеддингами всех токенов, мы можем использовать только одно объединенное представление, что снижает размерность данных и, следовательно, вычислительные затраты.

In [14]:
if add_pooling_layer:
    word_embedding_model = models.Transformer(base_model, max_seq_length=512)
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode='mean')
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
else:
    model = SentenceTransformer(base_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Запустим модель на паре текстов, посмотрим что она выдает

In [26]:
embedding_1_example = model.encode([sentence_1_example], convert_to_tensor=True)
embedding_2_example = model.encode([sentence_2_example], convert_to_tensor=True)

In [31]:
embedding_1_example.shape, embedding_2_example.shape

(torch.Size([1, 768]), torch.Size([1, 768]))

Как мы видим, мы получили те самые эмбеддинги о которых и говорили. Давайте теперь посчитаем их косинусную близость.

In [29]:
cosine_scores = util.cos_sim(embedding_1_example, embedding_2_example)

In [30]:
cosine_scores

tensor([[0.8492]], device='cuda:0')

Далее инициализируем объет лосса. В данном случае мы будем использовать OnlineContrastiveLoss:

$$
\mathcal{L}_\text{cont}(\mathbf{x}_i, \mathbf{x}_j, \theta) = \mathbb{1}[y_i=y_j] \| f_\theta(\mathbf{x}_i) - f_\theta(\mathbf{x}_j) \|^2_2 + \mathbb{1}[y_i\neq y_j]\max(0, \epsilon - \|f_\theta(\mathbf{x}_i) - f_\theta(\mathbf{x}_j)\|_2)^2
$$

Если объяснять коротко, то дан список входных образцов $\mathbf{x}_i$, каждый из которых имеет соответствующую метку $y_i \in \{1, \dots, L\}$ среди $L$ классов. Мы хотим выучить функцию $f_\theta(.): \mathcal{X}\to\mathbb{R}^d$, которая кодирует образец в векторное представление таким образом, чтобы примеры из одного и того же класса имели схожие эмбеддинги, а примеры из разных классов имели очень разные. Таким образом, Contrastive Loss принимает пару входных данных и минимизирует расстояние между эмбеддингами, когда они принадлежат одному и тому же классу, но максимизирует это расстояние в противном случае.

При этом в данном случае, лосс вычисляется лишь на тех объектах, которые являются hard negative или hard positive:

1. hard negative - такие пары объектов, которые являются negative сэмплами и при этом их расстояние меньше средних расстояний негативных пар;
2. hard positive - такие пары объектов, которые являются positive сэмплами и при этом их расстояние в среднем больше расстояний позитивных пар объектов.

в нашем случае под позитивными парамы текстов мы подразумеваем те пары текстов, которые являются дубликатами, а под негативными - те пары текстов, которые дубликатами не являются

In [74]:
train_loss = losses.OnlineContrastiveLoss(
    model=model,
    distance_metric=loss_params['distance_metric'],
    margin=loss_params['margin']
)

Далее преобразуем наши словарь текстов с ключами:

1. sentence_1;
2. sentence_2;
3. label.

В список картежей, состоящих из двух целочисленных значений, которые являются идентификаторами текстов и представляют собой связь (ребра графа) между двумя текстами.

И создадим DataLoader, который будет учавствовать в обучении

In [22]:
label_id = 1 if 'same' == sentence_label_example else 0
test_input_example = InputExample(texts=[sentence_1_example,
                                         sentence_2_example],
                                  label=float(label_id))
print(test_input_example)

<InputExample> label: 1.0, texts: New York, March 7.-—(AP)--The
tueboat Joyce Card blew up as she
was moving out of Erie Basin,
ttrooklyn, today, killing two men
and injuring three. Three others
are missing.

The boat, owned by the Card
Towing Line, Inc., had just been
overhauled and was starting out on
her first assignment when a blast
in the boiler room sent her to the
bottom.

The three men rescued were on
the deck at the time of the ex-
plosion, They were picked up by
rescuers who dived from other tugs
nearby, The rest of the crew was
below deck and it was believed
certain that all died. Two bodies
were recovered,
 ; NEW YORK, March T—(AP)—
The tugboat, Joyce Card, blew up
taday, killing two men and injur-
ing three. Three others are miss-
fy.

A blast in the boiler room sent
her Lo the bottom.

The three men reseued were on
the deck at the time of the explo-
sion. They were picked tp hy res-
utys. The r of the crew was
below and it was helieved all were
dead. Two bodies were recov

In [ ]:
def load_data_as_pairs(data, type):

    sentence_1_list = data['sentence_1']
    sentence_2_list = data['sentence_2']
    labels = data['labels']

    label2int = {"same": 1, "different": 0, 1: 1, 0: 0}

    paired_data = []
    for i in range(len(sentence_1_list)):
        label_id = label2int[labels[i]]
        paired_data.append(InputExample(texts=[sentence_1_list[i], sentence_2_list[i]], label=float(label_id)))

    print(f'{len(paired_data)} {type} pairs')

    return paired_data

In [ ]:
train_samples = load_data_as_pairs(train_data, type="neural")
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)

# Evaluate with multiple evaluators
dev_pairs = load_data_as_pairs(dev_data, type="dev")

После чего инициализируем объекты, которые будут выполнять оценку

In [ ]:
evaluators = [
    evaluation.BinaryClassificationEvaluator.from_input_examples(dev_pairs),
    clu_evaluators.ClusterEvaluator.from_input_examples(dev_pairs, cluster_type="agglomerative")
]

seq_evaluator = evaluation.SequentialEvaluator(evaluators, main_score_function=lambda scores: scores[-1])


И теперь уже мы подошли к этапу, когда можем запускать обучение.

In [75]:
if False:
    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        evaluator=seq_evaluator,
        epochs=num_epochs,
        warmup_steps=math.ceil(len(train_dataloader) * warmup_epochs),
        output_path=model_save_path,
        evaluation_steps=112,
        checkpoint_save_steps=112,
        checkpoint_path=model_save_path,
        save_best_model=True,
        checkpoint_save_total_limit=10
    )

Но для удобства давайте объединим всю эту история в одну функцию, которую далее будем использовать для обучения:

In [14]:
def train_biencoder(
        train_data: dict = None,
        dev_data: dict = None,
        base_model='sentence-transformers/all-MiniLM-L12-v2',
        add_pooling_layer=False,
        train_batch_size=64,
        num_epochs=10,
        warmup_epochs=1,
        loss_params=None,
        model_save_path="output",
):

    os.makedirs(model_save_path, exist_ok=True)

    # Base language model
    if add_pooling_layer:
        word_embedding_model = models.Transformer(base_model, max_seq_length=512)
        pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode='mean')
        model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    else:
        model = SentenceTransformer(base_model)

    train_loss = losses.OnlineContrastiveLoss(
        model=model,
        distance_metric=loss_params['distance_metric'],
        margin=loss_params['margin']
    )

    train_samples = data_loaders.load_data_as_pairs(train_data, type="neural")
    train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
    # Evaluate with multiple evaluators
    dev_pairs = data_loaders.load_data_as_pairs(dev_data, type="dev")

    evaluators = [
        evaluation.BinaryClassificationEvaluator.from_input_examples(dev_pairs),
        clu_evaluators.ClusterEvaluator.from_input_examples(dev_pairs, cluster_type="agglomerative")
    ]

    seq_evaluator = evaluation.SequentialEvaluator(evaluators, main_score_function=lambda scores: scores[-1])

    logger.info("Evaluate model without neural")
    seq_evaluator(model, epoch=0, steps=0, output_path=model_save_path)

    # Train the model
    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        evaluator=seq_evaluator,
        epochs=num_epochs,
        warmup_steps=math.ceil(len(train_dataloader) * warmup_epochs),
        output_path=model_save_path,
        evaluation_steps=112,
        checkpoint_save_steps=112,
        checkpoint_path=model_save_path,
        save_best_model=True,
        checkpoint_save_total_limit=10
    )


In [ ]:
train_biencoder(
    train_data=train_data,
    dev_data=dev_data,
    base_model='sentence-transformers/all-mpnet-base-v2',
    add_pooling_layer=False,
    train_batch_size=8,
    num_epochs=2,
    warmup_epochs=2,
    loss_params={'distance_metric': losses.SiameseDistanceMetric.COSINE_DISTANCE, 'margin': 0.2},
    model_save_path=f'output/{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}',
)

# Напишем функцию для тренировки cross-encoder **(Не трогаем)**

Давайте шаг за шагом пройдем весь путь.

In [ ]:
model_name='roberta-base'
lr=2e-05
train_batch_size=32
num_epochs=5
warm_up_perc=0.2
eval_per_epoch=10
model_save_path=f'output/{datetime.now().strftime("%Y-%m-%d_%H-%M")}'

Для начала нам необходимо инициализировать модель CrossEncoder, которая будет выдавать нам одно значение - а именно скор, который замеряет то, является ли модель дубликатом или нет. Как вы уже можете знать, кросс энкодер имеет примерно следующую структуру:

![Cross-Encoder+BiEncoder](https://raw.githubusercontent.com/UKPLab/sentence-transformers/master/docs/img/Bi_vs_Cross-Encoder.png)

In [ ]:
model = CrossEncoder(model_name, num_labels=1)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


2024-05-27 11:26:48 - Use pytorch device: mps


Прежде чем работать с DataLoaders, нам необходимо обернуть данные в формат SentenceTransformers

In [ ]:
train = data_loaders.load_data_as_pairs(train_data, type="neural")
dev = data_loaders.load_data_as_pairs(dev_data, type="dev")

73928 neural pairs
6288 dev pairs


После чего необходимо подготовить даталоадеры, которые будут использоваться для батчевой подачи данных при обучении. Мы не будем останавливаться на конкретной реализации, но вы можете посмотреть ее в скрипте [data_loaders](./data_loaders.py).

In [ ]:
train_dataloader = DataLoader(train, shuffle=True, batch_size=train_batch_size)

После чего нам необходимо определить объекты, выполняющие оценку качества примеров.
В нашем случае, помимо базовых оценок, необходимо производить оценку при помощи класстеров, по-скольку мы считаем все объекты, попавшие в один класстер - дубликатами и нам важно строить клики на основе скоров, которые выдает кросс енкодер

In [ ]:
evaluators = [
    CEBinaryClassificationEvaluator.from_input_examples(dev, name='dev'),
    clu_evaluators.CEClusterEvaluator.from_input_examples(dev, name='dev'),
]

seq_evaluator = evaluation.SequentialEvaluator(evaluators, main_score_function=lambda scores: scores[0])

In [ ]:
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * warm_up_perc)
logger.info("Warmup-steps: {}".format(warmup_steps))

Теперь, когда все готово, мы можем запустить тренировку модели, используя код из следующей ячейки.

In [ ]:
if True:
    model.fit(train_dataloader=train_dataloader,
            evaluator=seq_evaluator,
            epochs=num_epochs,
            evaluation_steps=int(len(train_dataloader)*(1/eval_per_epoch)),
            loss_fct=torch.nn.BCEWithLogitsLoss(),
            optimizer_params={"lr": lr},
            warmup_steps=warmup_steps,
            output_path=model_save_path)

Но для удобства, давайте объединим все наши разрозненные куски кода в одну единую функцию

In [ ]:
def train_crossencoder(
        train_data: Dict[str, List],
        dev_data: Dict[str, List],
        model_name: str,
        lr,
        train_batch_size,
        num_epochs,
        warm_up_perc,
        eval_per_epoch,
        model_save_path,
):
    model = CrossEncoder(model_name, num_labels=1)

    train = data_loaders.load_data_as_pairs(train_data, type="neural")
    dev = data_loaders.load_data_as_pairs(dev_data, type="dev")

    # Wrap train_samples, which is a list of InputExample, in a pytorch DataLoader
    train_dataloader = DataLoader(train, shuffle=True, batch_size=train_batch_size)

    # Evaluate with multiple evaluators
    evaluators = [
        CEBinaryClassificationEvaluator.from_input_examples(dev, name='dev'),
        clu_evaluators.CEClusterEvaluator.from_input_examples(dev, name='dev'),
    ]

    seq_evaluator = evaluation.SequentialEvaluator(evaluators, main_score_function=lambda scores: scores[0])

    warmup_steps = math.ceil(len(train_dataloader) * num_epochs * warm_up_perc)
    logger.info("Warmup-steps: {}".format(warmup_steps))

    # Train the model
    model.fit(train_dataloader=train_dataloader,
              evaluator=seq_evaluator,
              epochs=num_epochs,
              evaluation_steps=int(len(train_dataloader)*(1/eval_per_epoch)),
              loss_fct=torch.nn.BCEWithLogitsLoss(),
              optimizer_params={"lr": lr},
              warmup_steps=warmup_steps,
              output_path=model_save_path)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


KeyboardInterrupt: 

# Оценим качество решения c использованием bi-encoder модели

Преобразуем наши лейбли в целочисленный формат, где:
1. ```same``` - 1;
2. ```different``` - 0.

In [58]:
labels = [1 if label == 'same' else 0 for label in dev_data['labels']]

Инициализируем класс оценщика, который будет сравнивать кластера

In [60]:
evaluator = clu_evaluators.ClusterEvaluator(
    sentences1=dev_data['sentence_1'],
    sentences2=dev_data['sentence_2'],
    labels=labels,
    name='',
    batch_size=512,
    show_progress_bar=False,
    write_csv=True,
    cluster_type="agglomerative"
)

Инициализируем модель, метрики которой будем валидировать

In [61]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
metrics = evaluator.compute_metrices(
    model
)

Такие метрики у нас получились на непредобученной модели ```sentence-transformers/all-mpnet-base-v2```

In [68]:
pd.Series(metrics)

accuracy              0.971731
accuracy_threshold    0.210000
f1                    0.829710
f1_threshold          0.210000
precision             0.767169
recall                0.903353
dtype: float64

Давайте немного объясним метрики:

# Самостоятельная работа

Попробуйте взять вот этот датасет из корня [./duplicated_news.csv](./duplicated_news.csv)

1. Предобработать его;
2. Померить качество с использованием модели, например, [cointegrated/LaBSE-en-ru](https://huggingface.co/cointegrated/LaBSE-en-ru);
3. Дообучить ее и посмотреть на результаты, которые получаются.

# Ссылки

Если хотите узнать больше про Contrastive Learning, то можете смело обращаться вот сюда:

1. [Contrastive](https://lilianweng.github.io/posts/2021-05-31-contrastive/) - блок от ресерчера в OpenAI, в котором есть очень много интересных вещей помимо contrastive learning. Там и про трансформеры и про диффузионки и еще много всего - поэтому маст хев, если вы интересуетесь deep learning;